In [3]:

import numpy as np
import pandas as pd
# import lightgbm as lgb
import gc

In [29]:
train = pd.read_csv('train_2016.csv')
prop = pd.read_csv('properties_2016.csv')

for c, dtype in zip(prop.columns, prop.dtypes):	
    if dtype == np.float64:		
        prop[c] = prop[c].astype(np.float32)

df_train = train.merge(prop, how='left', on='parcelid')

x_train = df_train.drop(['parcelid', 'logerror', 'transactiondate', 'propertyzoningdesc', 'propertycountylandusecode'], axis=1)
y_train = df_train['logerror'].values
print(x_train.shape, y_train.shape)

train_columns = x_train.columns

for c in x_train.dtypes[x_train.dtypes == object].index.values:
    x_train[c] = (x_train[c] == True)

del df_train; gc.collect()

split = 90000
# x_train, y_train, x_valid, y_valid = x_train[:split], y_train[:split], x_train[split:], y_train[split:]
# x_train = x_train.values.astype(np.float32, copy=False)
# x_valid = x_valid.values.astype(np.float32, copy=False)

/Users/Leslie/anaconda3/lib/python3.5/site-packages/IPython/core/interactiveshell.py:2717: DtypeWarning: Columns (22,32,34,49,55) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


(90811, 55) (90811,)


In [9]:
from hyperopt import fmin, tpe, hp, STATUS_OK, Trials
from sklearn.model_selection import cross_val_score
import xgboost as xgb
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import make_scorer

In [17]:
scorer_mae = make_scorer(mean_absolute_error, greater_is_better=False)
def hyperopt_train_test(params):
    t = params['type']
    del params['type']
    if t == 'xgb':
        reg = xgb.XGBRegressor(**params)
    else:
        return 0, 0
    cv_score = cross_val_score(reg, x_train, y_train, scoring=scorer_mae, cv=5)
    score_mean = cv_score.mean()
    score_std = cv_score.std()
    return score_mean, score_std

In [18]:
space = hp.choice('classifier_type', [
    {
        'type': 'xgb',
        'learning_rate': hp.uniform('learning_rate', 0.02, 0.1),
        'min_child_weight': hp.choice('min_child_weight', range(30, 60)),
        'max_depth': hp.choice('max_depth', range(5, 10)),
        'subsample': hp.uniform('subsample', 0.2, 0.4),
        'n_estimators': hp.choice('n_estimators', range(150, 200)),
#         'colsample_bytree': hp.uniform('colsample_bytree', 0.4, 0.6),
#         'reg_lambda': hp.uniform('reg_lambda', 1, 2),
#         'reg_alpha': hp.uniform('reg_alpha', 1, 2)
}])

In [19]:
def f(params):
    global best, count
    count += 1
    result = hyperopt_train_test(params.copy())
    acc = result[0]
    std = result[1]
    print('iters:', count, ', acc:', acc, 'std:', std, '\nusing', params, '\n')
    return {'loss': -acc, 'status': STATUS_OK}

In [20]:
count = 0 
trials = Trials()
best = fmin(f, space, algo=tpe.suggest, max_evals=50, trials=trials)

iters: 1 , acc: -0.0696628355945 std: 0.0031577906436 
using {'n_estimators': 161, 'learning_rate': 0.04341351063395084, 'subsample': 0.2011198373732315, 'max_depth': 6, 'min_child_weight': 46, 'type': 'xgb'} 

iters: 2 , acc: -0.0697849350389 std: 0.00308450170629 
using {'n_estimators': 170, 'learning_rate': 0.06667470420025107, 'subsample': 0.2787160545105113, 'max_depth': 5, 'min_child_weight': 54, 'type': 'xgb'} 

iters: 3 , acc: -0.0697717012101 std: 0.00308162079381 
using {'n_estimators': 163, 'learning_rate': 0.06188881725110355, 'subsample': 0.35630203437981156, 'max_depth': 5, 'min_child_weight': 33, 'type': 'xgb'} 

iters: 4 , acc: -0.0700098481565 std: 0.00307315404674 
using {'n_estimators': 185, 'learning_rate': 0.04554641431545384, 'subsample': 0.24986176061757903, 'max_depth': 8, 'min_child_weight': 40, 'type': 'xgb'} 

iters: 5 , acc: -0.0695535507155 std: 0.00309841666333 
using {'n_estimators': 173, 'learning_rate': 0.0416198137601932, 'subsample': 0.307746800486719

KeyboardInterrupt: 

In [30]:
params = {'n_estimators': 173, 'learning_rate': 0.0416198137601932, 
          'subsample': 0.3077468004867197, 
          'max_depth': 6, 'min_child_weight': 58,}
reg = xgb.XGBRegressor()
reg.set_params(**params)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.0416198137601932, max_delta_step=0, max_depth=6,
       min_child_weight=58, missing=None, n_estimators=173, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True,
       subsample=0.3077468004867197)

In [31]:
reg.fit(x_train, y_train)

XGBRegressor(base_score=0.5, colsample_bylevel=1, colsample_bytree=1, gamma=0,
       learning_rate=0.0416198137601932, max_delta_step=0, max_depth=6,
       min_child_weight=58, missing=None, n_estimators=173, nthread=-1,
       objective='reg:linear', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=0, silent=True,
       subsample=0.3077468004867197)

In [32]:
sample = pd.read_csv('sample_submission.csv')
sample['parcelid'] = sample['ParcelId']
df_test = sample.merge(prop, on='parcelid', how='left')

x_test = df_test[train_columns]

for c in x_test.dtypes[x_test.dtypes == object].index.values:
    x_test[c] = (x_test[c] == True)
# x_test = x_test.values.astype(np.float32, copy=False)

/Users/Leslie/anaconda3/lib/python3.5/site-packages/IPython/kernel/__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [33]:
p_test = reg.predict(x_test)

In [34]:
p_test = 0.97*p_test + 0.03*0.011
sub = pd.read_csv('sample_submission.csv')
for c in sub.columns[sub.columns != 'ParcelId']:
    sub[c] = p_test

sub.to_csv('result.csv', index=False, float_format='%.8f')